# Calculate biomass from deadwood

# Imports and Set-up

In [1]:
# Standard Imports
import sys
import pandas as pd
import numpy as np

# Google Cloud Imports
import pandas_gbq

In [2]:
# Util imports
sys.path.append("../../")  # include parent directory
from src.settings import (
    GCP_PROJ_ID,
    CARBON_POOLS_OUTDIR,
    SPECIES_LOOKUP_CSV,
    PC_PLOT_LOOKUP_CSV,
    TMP_OUT_DIR,
)

from src.biomass_equations import (
    vmd0001_eq1,
    vmd0002_eq1,
    vmd0002_eq2,
    vmd0002_eq3,
    vmd0002_eq4,
    vmd0002_eq7,
    vmd0002_eq8,
    get_solid_diamter,
    calculate_tree_height,
    allometric_tropical_tree,
    allometric_peatland_tree,
)

In [3]:
# Variables
PLOT_INFO_CSV = CARBON_POOLS_OUTDIR / "plot_info.csv"
STUMPS_CSV = CARBON_POOLS_OUTDIR / "stumps.csv"
LDW_CSV = CARBON_POOLS_OUTDIR / "lying_deadwood_wo_hollow.csv"
LDW_HOLLOW_CSV = CARBON_POOLS_OUTDIR / "lying_deadwood_hollow.csv"
DEAD_TREES_CSV = CARBON_POOLS_OUTDIR / "dead_trees.csv"

# Temporary Output Files
tmp_dead_trees_c1 = TMP_OUT_DIR / "c1_dead_trees.csv"
tmp_dead_trees_c1_wd = TMP_OUT_DIR / "c1_dead_trees_wd.csv"

# BigQuery Variables
SRC_DATASET_ID = "biomass_inventory"
DATASET_ID = "carbon_stock"
IF_EXISTS = "replace"

# Processing Conditions
OUTLIER_REMOVAL = "get_ave"  # Options: "get_ave", "drop_outliers", "eq_150"

## Load data

### Plot Data

In [4]:
if PLOT_INFO_CSV.exists():
    plot_info = pd.read_csv(PLOT_INFO_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.plot_info"""

    # Read the BigQuery table into a dataframe
    plot_info = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    plot_info.to_csv(PLOT_INFO_CSV, index=False)

In [5]:
plot_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 674 entries, 0 to 673
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   unique_id                  674 non-null    object 
 1   data_recorder              674 non-null    object 
 2   team_no                    674 non-null    int64  
 3   plot_code_nmbr             674 non-null    int64  
 4   plot_type                  674 non-null    object 
 5   sub_plot                   674 non-null    object 
 6   yes_no                     674 non-null    object 
 7   sub_plot_shift             634 non-null    object 
 8   GPS_waypt                  634 non-null    float64
 9   GPS_id                     634 non-null    float64
 10  GPS                        577 non-null    object 
 11  GPS_latitude               577 non-null    float64
 12  GPS_longitude              577 non-null    float64
 13  GPS_altitude               577 non-null    float64

### Stumps

In [6]:
if STUMPS_CSV.exists():
    stumps = pd.read_csv(STUMPS_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{DATASET_ID}.stumps"""

    # Read the BigQuery table into a dataframe
    stumps = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    stumps.to_csv(STUMPS_CSV, index=False)

In [7]:
stumps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1754 entries, 0 to 1753
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   unique_id      1754 non-null   object 
 1   nest           1754 non-null   int64  
 2   Diam1          1754 non-null   float64
 3   Diam2          1754 non-null   float64
 4   slope          1754 non-null   float64
 5   height         1754 non-null   float64
 6   cut_cl         1754 non-null   object 
 7   hollow_go      1754 non-null   object 
 8   hollow_d1      171 non-null    float64
 9   hollow_d2      171 non-null    float64
 10  stump_density  1754 non-null   float64
dtypes: float64(7), int64(1), object(3)
memory usage: 150.9+ KB


### Lying deadwood

In [8]:
if LDW_CSV.exists():
    ldw = pd.read_csv(LDW_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.lying_deadwood_wo_hollow"""

    # Read the BigQuery table into a dataframe
    ldw = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ldw.to_csv(LDW_CSV, index=False)

In [9]:
ldw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1826 entries, 0 to 1825
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   1826 non-null   object 
 1   repetition  1826 non-null   int64  
 2   type        1826 non-null   object 
 3   class       1826 non-null   object 
 4   diameter    1826 non-null   float64
 5   density     1826 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 85.7+ KB


In [10]:
if LDW_HOLLOW_CSV.exists():
    ldw_hollow = pd.read_csv(LDW_HOLLOW_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.lying_deadwood_hollow"""

    # Read the BigQuery table into a dataframe
    ldw_hollow = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    ldw_hollow.to_csv(LDW_HOLLOW_CSV, index=False)

In [11]:
ldw_hollow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   unique_id   15 non-null     object 
 1   repetition  15 non-null     int64  
 2   type        15 non-null     object 
 3   class       15 non-null     object 
 4   hollow_d1   15 non-null     float64
 5   hollow_d2   15 non-null     float64
 6   diameter    15 non-null     float64
 7   density     15 non-null     float64
dtypes: float64(4), int64(1), object(3)
memory usage: 1.1+ KB


### Standing Deadwood

In [12]:
if DEAD_TREES_CSV.exists():
    dead_trees = pd.read_csv(DEAD_TREES_CSV)
else:
    query = f"""
    SELECT
        * 
    FROM {GCP_PROJ_ID}.{SRC_DATASET_ID}.dead_trees"""

    # Read the BigQuery table into a dataframe
    dead_trees = pandas_gbq.read_gbq(query, project_id=GCP_PROJ_ID)
    dead_trees.to_csv(DEAD_TREES_CSV, index=False)

In [13]:
dead_trees.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 256 entries, 0 to 255
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   unique_id     256 non-null    object 
 1   nest          256 non-null    int64  
 2   species_name  203 non-null    float64
 3   DBH_cl1       2 non-null      float64
 4   class         256 non-null    int64  
 5   subclass      254 non-null    object 
 6   family_name   12 non-null     float64
 7   dbh_tall      254 non-null    float64
 8   db_tall       254 non-null    float64
 9   tall_density  254 non-null    float64
 10  slope_t_tall  254 non-null    float64
 11  slope_b_tall  254 non-null    float64
 12  dist_t_tall   254 non-null    float64
dtypes: float64(9), int64(2), object(2)
memory usage: 26.1+ KB


### Tree species

In [14]:
species = pd.read_csv(SPECIES_LOOKUP_CSV)

In [15]:
species.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 375 entries, 0 to 374
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   code_family      74 non-null     float64
 1   family           74 non-null     object 
 2   scientific_name  375 non-null    object 
 3   local_name       375 non-null    object 
 4   code_species     375 non-null    int64  
 5   corrected_genus  375 non-null    object 
 6   wood_density     375 non-null    float64
dtypes: float64(2), int64(1), object(4)
memory usage: 20.6+ KB


In [16]:
species.head(2)

,code_family,family,scientific_name,local_name,code_species,corrected_genus,wood_density
0,999.0,Unknown,Litchi chinensis,Alupag - amo,193,Litchi,0.608902
1,1.0,Alangiaceae,Alangium javanicum,Putian,15,Alangium,0.608902


In [17]:
species_dict = (
    species[["scientific_name", "code_species"]]
    .set_index("code_species")
    .to_dict()["scientific_name"]
)

In [18]:
# create lookup table for family name and code
species_family = species[["code_family", "family"]].drop_duplicates()

In [19]:
family_dict = species_family.set_index("code_family").to_dict()["family"]

### Plot lookup

In [20]:
plot_strata = pd.read_csv(PC_PLOT_LOOKUP_CSV)

In [21]:
plot_strata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1760 entries, 0 to 1759
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Strata     1760 non-null   int64 
 1   unique_id  1760 non-null   object
 2   LCC        1760 non-null   object
 3   Bgy_Name   1760 non-null   object
 4   Mun_Name   1760 non-null   object
 5   Pro_Name   1760 non-null   object
dtypes: int64(1), object(5)
memory usage: 82.6+ KB


# Calculate stump biomass

In [22]:
stumps.head(2)

,unique_id,nest,Diam1,Diam2,slope,height,cut_cl,hollow_go,hollow_d1,hollow_d2,stump_density
0,308C1,2,30.0,29.0,43.0,18.0,saw_axe,no,NaN,NaN,1.0
1,249B1,2,15.0,10.0,51.0,80.0,saw_axe,no,NaN,NaN,3.0


In [23]:
# get wood density equivalent for each density class

density_val = {1: 0.54, 2: 0.35, 3: 0.21}
stumps["stump_density_val"] = stumps["stump_density"].replace(density_val).fillna(0.21)

In [24]:
# Get biomass for each stump
stumps = vmd0002_eq2(stumps, "Diam1", "Diam2", "height", "stump_density_val")

In [25]:
# Get biomass of each stump that is hollow
stumps_hollow = vmd0002_eq2(
    stumps, "hollow_d1", "hollow_d2", "height", "stump_density_val"
)

In [26]:
# Get biomass to subtract due to hollow stumps
stumps["tonnes_dry_matter_hollow"] = stumps_hollow["tonnes_dry_matter"]

In [27]:
# Subtract biomass of hollow stumps from total biomass
stumps["tonnes_dry_matter"] = np.where(
    (~stumps["tonnes_dry_matter_hollow"].isna())
    & (stumps["tonnes_dry_matter_hollow"] > 0),
    stumps["tonnes_dry_matter"] - stumps["tonnes_dry_matter_hollow"],
    stumps["tonnes_dry_matter"],
)

In [28]:
# Remove biomass_hollow column to avoid confusion
stumps.drop(columns=["tonnes_dry_matter_hollow"], inplace=True)

In [29]:
stumps.head(2)

,unique_id,nest,Diam1,Diam2,slope,height,cut_cl,hollow_go,hollow_d1,hollow_d2,stump_density,stump_density_val,tonnes_dry_matter
0,308C1,2,30.0,29.0,43.0,18.0,saw_axe,no,NaN,NaN,1.0,0.54,2.8674
1,249B1,2,15.0,10.0,51.0,80.0,saw_axe,no,NaN,NaN,3.0,0.21,2.1000


In [30]:
# get sum opf dry matter per subplot
stumps_agg = vmd0002_eq3(stumps, ["unique_id", "nest"], "tonnes_dry_matter")

In [31]:
# get the slope adjusted area per nest per subplot
plot_info_subset = plot_info[
    [
        "unique_id",
        "corrected_plot_area_n2_m2",
        "corrected_plot_area_n3_m2",
        "corrected_plot_area_n4_m2",
    ]
].copy()
plot_info_subset.dropna(inplace=True)
plot_info_subset.drop_duplicates(subset=["unique_id"], inplace=True)
plot_info_subset_dict = plot_info_subset.to_dict(orient="records")

In [32]:
# add the correct area using the unique_id and nest number
stumps_agg["corrected_area_m2"] = stumps_agg.apply(
    lambda x: next(
        (
            item["corrected_plot_area_n" + str(x["nest"]) + "_m2"]
            for item in plot_info_subset_dict
            if item["unique_id"] == x["unique_id"]
        ),
        None,
    ),
    axis=1,
)

In [33]:
# convert square meters to hectares
stumps_agg["corrected_area_ha"] = stumps_agg["corrected_area_m2"] / 10_000

In [34]:
stumps_agg = vmd0002_eq4(stumps_agg, "tonnes_dry_matter", "corrected_area_ha")

In [35]:
stumps_agg.rename(
    columns={"tonnes_dry_matter_ha": "stumps_tonnes_dry_matter_ha"}, inplace=True
)

## Get total stump biomass per hectare per subplot

In [36]:
stumps_agg

,unique_id,nest,tonnes_dry_matter,corrected_area_m2,corrected_area_ha,stumps_tonnes_dry_matter_ha
0,100C1,3,11.40300,707.141090,0.070714,161.254948
1,100D1,3,11.30700,779.240642,0.077924,145.102801
2,100D1,4,13.59320,1385.316697,0.138532,98.123411
3,101A1,2,1.95300,90.485722,0.009049,215.835156
4,101B1,3,11.85800,711.382240,0.071138,166.689570
...,...,...,...,...,...,...
706,99B2,2,23.89135,122.718463,0.012272,1946.842342
707,99B2,3,3.35475,1104.466167,0.110447,30.374403
708,99B2,4,14.85960,1963.495408,0.196350,75.679321
709,99D2,3,109.05255,803.627255,0.080363,1357.004125


# Calculate Lying deadwood biomass

In [37]:
ldw.head(2)

,unique_id,repetition,type,class,diameter,density
0,308D2,1,tr1,FC,16.5,3.0
1,308D2,2,tr1,FC,18.3,3.0


## No hollow

### Outlier removal

In [38]:
ldw.describe()

,repetition,diameter,density
count,1826.000000,1826.000000,1826.000000
mean,2.529573,27.733866,1.937021
std,1.943715,109.834565,0.744572
min,1.000000,10.000000,1.000000
25%,1.000000,14.425000,1.000000
50%,2.000000,20.000000,2.000000
75%,3.000000,30.000000,2.000000
max,15.000000,4592.000000,3.000000


In [39]:
# Filter the ldw DataFrame to keep rows where diameter is less than or equal to the 98th percentile
ldw = ldw[ldw["diameter"] <= 150]

In [40]:
ldw = vmd0002_eq7(ldw, "diameter")

In [41]:
ldw = vmd0002_eq8(ldw, "density")

In [42]:
ldw.head(2)

,unique_id,repetition,type,class,diameter,density,deadwood_volume,tonnes_dry_matter
0,308D2,1,tr1,FC,16.5,3.0,3.35875,0.705337
1,308D2,2,tr1,FC,18.3,3.0,4.13154,0.867623


## Get total lyind deadwood biomass from 

## Hollow Lying Deadwood

In [43]:
ldw_hollow = get_solid_diamter(ldw_hollow, "hollow_d1", "hollow_d2", "diameter")

In [44]:
ldw_hollow = vmd0002_eq7(ldw_hollow, "solid_diameter")

In [45]:
ldw_hollow = vmd0002_eq8(ldw_hollow, "density")

In [46]:
ldw_hollow.head(2)

,unique_id,repetition,type,class,hollow_d1,hollow_d2,diameter,density,solid_diameter,deadwood_volume,tonnes_dry_matter
0,249D1,1,tr2,MDF,12.0,10.0,16.7,3.0,12.565429,1.947890,0.409057
1,290A1,2,tr2,MCB,68.0,28.0,62.0,1.0,39.242834,18.998988,10.259454


# Calculate Standing Deadwood Biomass

## Calculate biomass for Class 1 standing deadwood trees
Class 1 standing dead trees that is fresh and can be treated as living trees in terms of biomass. The method applied mimics the process from living trees

In [ ]:
dead_trees.head(2)

In [ ]:
dead_trees.rename(
    columns={"family_name": "code_family", "species_name": "code_species"}, inplace=True
)

In [ ]:
c1_dead_trees = dead_trees.loc[dead_trees["class"] == 1].copy()

In [ ]:
c1_dead_trees["family_name"] = c1_dead_trees["code_family"].replace(family_dict)

In [ ]:
c1_dead_trees["scientific_name"] = c1_dead_trees["code_species"].replace(species_dict)

In [ ]:
c1_dead_trees.to_csv(tmp_dead_trees_c1)

In [ ]:
c1_dead_trees

### Get genus and wood density using BIOMASS R Library

In [ ]:
!Rscript $SRC_DIR"/get_wood_density.R" $tmp_dead_trees_c1 $tmp_dead_trees_c1_wd

In [ ]:
c1_dead_trees = pd.read_csv(tmp_dead_trees_c1_wd)

In [ ]:
c1_dead_trees

In [ ]:
c1_dead_trees = calculate_tree_height(c1_dead_trees, "DBH_cl1")

In [ ]:
c1_dead_trees = c1_dead_trees.merge(
    plot_strata[["unique_id", "Strata"]], on="unique_id", how="left"
)

In [ ]:
c1_dead_trees_tropical = c1_dead_trees.loc[
    c1_dead_trees["Strata"].isin([1, 2, 3])
].copy()

In [ ]:
c1_dead_trees_tropical = allometric_tropical_tree(
    c1_dead_trees_tropical, "wood_density", "DBH_cl1", "height"
)

In [ ]:
c1_dead_trees_peatland = c1_dead_trees.loc[
    c1_dead_trees["Strata"].isin([4, 5, 6])
].copy()

In [ ]:
c1_dead_trees_peatland = allometric_peatland_tree(c1_dead_trees_peatland, "DBH_cl1")

In [ ]:
c1_dead_trees = pd.concat([c1_dead_trees_tropical, c1_dead_trees_peatland])

In [ ]:
c1_dead_trees = vmd0001_eq1(c1_dead_trees, 0.47)

In [ ]:
c1_dead_trees.drop(columns=["X"], inplace=True)

In [ ]:
c1_dead_trees

## Calculate biomass for Class 2  standing deadwood tall trees

Class 2 are standing dead trees with assigned density class

In [ ]:
c2_dead_trees_t = dead_trees.loc[
    (dead_trees["class"] == 2) & (dead_trees["subclass"] == "tall")
].copy()

In [ ]:
# convert slope to radians
c2_dead_trees_t["slope_t_tall_radians"] = np.atan(c2_dead_trees_t["slope_t_tall"]) / 100
c2_dead_trees_t["slope_b_tall_radians"] = np.atan(c2_dead_trees_t["slope_b_tall"]) / 100

In [ ]:
# estimate tree height
c2_dead_trees_t = calculate_tree_height(
    c2_dead_trees_t,
    trig_leveling=True,
    dist_col="dist_t_tall",
    slope_b_col="slope_b_tall_radians",
    slope_t_col="slope_t_tall_radians",
)

In [ ]:
# set wood density equivalent for each density class
density_val = {1: 0.54, 2: 0.35, 3: 0.21}
c2_dead_trees_t["density_val"] = (
    c2_dead_trees_t["tall_density"].replace(density_val).fillna(0.21)
)

In [ ]:
c2_dead_trees_t = vmd0002_eq1(c2_dead_trees_t, "db_tall", "height", "density_val")

In [ ]:
c2_dead_trees_t